<a href="https://colab.research.google.com/github/VishnuSriraj/Data-Influence/blob/main/InfluenceScores_l2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import pandas as pd
import scipy
import math
import copy
from torch.autograd import grad as gradient

In [ ]:
class InfluenceFunction(object):
	def __init__(self, model, X_train, y_train, loss_func, layer_index):
		if not isinstance(model, torch.nn.Module):
			raise RuntimeError(f"Only torch.nn.Module models are supported, got f{type(model)}")
		if not isinstance(X_train, torch.Tensor):
			raise RuntimeError(f"X_train must be <torch.Tensor>, got f{type(X_train)}")
		if not isinstance(y_train, torch.Tensor):
			raise RuntimeError(f"y_train must be <torch.Tensor>, got f{type(y_train)}")

		try:
			layers = model.layers
		except:
			raise RuntimeError('The model must have an attribute "layers".')

		self._model = copy.deepcopy(model)
		self._X_train = X_train
		self._y_train = y_train
		self._loss_func = loss_func
		self._layer_index = layer_index
		
		self._total_training_loss_grad = None
		self._Hessian_matrix = self._calculate_Hessian_matrix()
		self._Hessian_inv = np.linalg.inv(self._Hessian_matrix)
		

	def _calculate_Hessian_matrix(self):

		X_train = self._X_train
		y_train = self._y_train
		model = self._model

		y_pred = model(X_train)
		loss = self._loss_func(y_pred, y_train)
		layer = model.layers[self._layer_index]

		if not isinstance(layer, torch.nn.Linear):
			raise RuntimeError(
				f"Only support layer type torch.nn.Linear, got f{type(layer)}."
			)

		weights = model.layers[self._layer_index].weight
		bias = model.layers[self._layer_index].bias

		grad_L_w_1 = gradient(
			loss, (weights, bias), retain_graph=True, create_graph=True
		)
		self._total_training_loss_grad = np.array(
			grad_L_w_1[0][0].tolist() + grad_L_w_1[1].tolist()
		)

		Hessian = []
		for i in range(0, grad_L_w_1[0].shape[1]):
			grad_L_w_2 = gradient(
				grad_L_w_1[0][0][i], (weights, bias), retain_graph=True
			)
			Hessian.append(grad_L_w_2[0][0].tolist() + grad_L_w_2[1].tolist())
		grad_L_w_2 = gradient(grad_L_w_1[1][0], (weights, bias), retain_graph=True)
		Hessian.append(grad_L_w_2[0][0].tolist() + grad_L_w_2[1].tolist())

		return np.array(Hessian)

	def influence_remove_single(self, index):
		return -self.influence_add_single(self._X_train[index], self._y_train[index])

	def influence_modify_single(self, index, new_x, new_y):
		return self.influence_add_single(new_x, new_y) + self.influence_remove_single(index)

	def influence_add_single(self, x, y):

		if not isinstance(x, torch.Tensor):
			raise RuntimeError(f"Added x must be <torch.Tensor>, got f{type(x)}")
		if not isinstance(y, torch.Tensor):
			raise RuntimeError(f"Added y must be <torch.Tensor>, got f{type(y)}")

		x = x.reshape(1, -1).detach()
		y = y.reshape(1, -1).detach()

		model = self._model

		y_pred = model(x)
		loss_single = self._loss_func(y_pred, y)

		weights = model.layers[self._layer_index].weight
		bias = model.layers[self._layer_index].bias

		grad = gradient(loss_single, (weights, bias), retain_graph=True)
		grad = np.array(grad[0][0].tolist() + grad[1].tolist())

		param_offset = - np.dot(self._Hessian_inv, grad)

		influence = np.dot(self._total_training_loss_grad, param_offset)

		return influence

In [ ]:
class TorchNNCore(torch.nn.Module):
    def __init__(
        self, inps, hiddens=[], bias=True, seed=None, hidden_activation=torch.nn.ReLU
    ):
        super(TorchNNCore, self).__init__()
        if seed is not None:
            torch.manual_seed(seed)
        struct = [inps] + hiddens + [1]
        self.layers = [] # This layer attribute is required under 
        for i in range(1, len(struct)):
            self.layers.append(
                torch.nn.Linear(
                    in_features=struct[i - 1], out_features=struct[i], bias=bias
                )
            )
            if i == len(struct) - 1:
                self.layers.append(torch.nn.Sigmoid())
            else:
                self.layers.append(hidden_activation())
        self.model = torch.nn.Sequential(*self.layers)

    def forward(self, x):
        output = self.model(x)
        return output

In [ ]:
# Prepare training & testing dataset
data1 = pd.read_csv('./adult.csv').to_numpy()
data2 = pd.read_csv('./broward.csv').to_numpy()
data3 = pd.read_csv('./hospital.csv').to_numpy()

X_train1 = torch.tensor(data1[:,:-1], dtype=torch.float)
y_train1 = torch.tensor(data1[:,-1].reshape(-1,1), dtype=torch.float)

X_train2 = torch.tensor(data2[:,:-1], dtype=torch.float)
y_train2 = torch.tensor(data2[:,-1].reshape(-1,1), dtype=torch.float)

X_train3 = torch.tensor(data3[:,:-1], dtype=torch.float)
y_train3 = torch.tensor(data3[:,-1].reshape(-1,1), dtype=torch.float)

print("\nData1: ",X_train1.shape, y_train1.shape)
print("\nData2: ",X_train2.shape, y_train2.shape)
print("\nData3: ",X_train3.shape, y_train3.shape)


Data1:  torch.Size([45222, 98]) torch.Size([45222, 1])

Data2:  torch.Size([7214, 8]) torch.Size([7214, 1])

Data3:  torch.Size([52778, 124]) torch.Size([52778, 1])


In [ ]:
# Specify loss function, define model and optimizer
loss_func = torch.nn.BCELoss()

model1 = TorchNNCore(inps=X_train1.shape[1], hiddens=[512, 256, 128], hidden_activation=torch.nn.LeakyReLU)
optim1 = torch.optim.Adam(model1.parameters(), lr=0.001, weight_decay=0.0001)
model2 = TorchNNCore(inps=X_train2.shape[1], hiddens=[512, 256, 128], hidden_activation=torch.nn.LeakyReLU)
optim2 = torch.optim.Adam(model2.parameters(), lr=0.001, weight_decay=0.0001)
model3 = TorchNNCore(inps=X_train3.shape[1], hiddens=[512, 256, 128], hidden_activation=torch.nn.LeakyReLU)
optim3 = torch.optim.Adam(model3.parameters(), lr=0.001, weight_decay=0.0001)

# Before using influence function, we show the structure of the model
print(model1)
print(model2)
print(model3)

# And we print the "layer" attribute, which is used to fetch the layers above
print("\nModel 1:")
for item in model1.layers:
    print(item)
print("\nModel 2:")
for item in model2.layers:
    print(item)
print("\nModel 3:")
for item in model3.layers:
    print(item)

TorchNNCore(
  (model): Sequential(
    (0): Linear(in_features=98, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=128, out_features=1, bias=True)
    (7): Sigmoid()
  )
)
TorchNNCore(
  (model): Sequential(
    (0): Linear(in_features=8, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=128, out_features=1, bias=True)
    (7): Sigmoid()
  )
)
TorchNNCore(
  (model): Sequential(
    (0): Linear(in_features=124, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Li

In [ ]:
sum_influences1 = np.zeros(X_train1.shape[0])
sum_influences2 = np.zeros(X_train2.shape[0])
sum_influences3 = np.zeros(X_train3.shape[0])

# # For 10 times averaged result
# for i in range(0,10):
y_train_np1 = y_train1.detach().numpy()
# y_train_np2 = y_train2.detach().numpy()
# y_train_np3 = y_train3.detach().numpy()

# %%timeit
# Training for 455 epochs
for epoch in range(0,455):
    optim1.zero_grad()
    y_pred1 = model1(X_train1)
    loss1 = loss_func(y_pred1, y_train1)
    loss1.backward()
    optim1.step()
    if epoch%10==0:
        y_pred_np1 = (y_pred1.detach().numpy()) > 0.5
        accuracy1 = sum(y_pred_np1 == y_train_np1)/y_train_np1.shape[0]
        print('Epoch = %d, loss1 = %.4f, accuracy=%.4f'%(epoch, loss1.tolist(), accuracy1))
print('\n')

# # Training for 1443 epochs
# for epoch in range(0,1443):
#     optim2.zero_grad()
#     y_pred2 = model2(X_train2)
#     loss2 = loss_func(y_pred2, y_train2)
#     loss2.backward()
#     optim2.step()
#     if epoch%10==0:
#         y_pred_np2 = (y_pred2.detach().numpy()) > 0.5
#         accuracy2 = sum(y_pred_np2 == y_train_np2)/y_train_np2.shape[0]
#         print('Epoch = %d, loss2 = %.4f, accuracy=%.4f'%(epoch, loss2.tolist(), accuracy2))
# print('\n')        

# # Training for 660 epochs
# for epoch in range(0,660):
#     optim3.zero_grad()
#     y_pred3 = model3(X_train3)
#     loss3 = loss_func(y_pred3, y_train3)
#     loss3.backward()
#     optim3.step()
#     if epoch%10==0:
#         y_pred_np3 = (y_pred3.detach().numpy()) > 0.5
#         accuracy3 = sum(y_pred_np3 == y_train_np3)/y_train_np3.shape[0]
#         print('Epoch = %d, loss3 = %.4f, accuracy=%.4f'%(epoch, loss3.tolist(), accuracy3))
# print('\n')

optim1.zero_grad()
# optim2.zero_grad()
# optim3.zero_grad()

infl1 = InfluenceFunction(
    model = model1, 
    X_train = X_train1, 
    y_train = y_train1, 
    loss_func = loss_func,
    layer_index = -2 
)
# infl2 = InfluenceFunction(
#     model = model2, 
#     X_train = X_train2, 
#     y_train = y_train2, 
#     loss_func = loss_func, 
#     layer_index = -2
# )
# infl3 = InfluenceFunction(
#     model = model3, 
#     X_train = X_train3, 
#     y_train = y_train3, 
#     loss_func = loss_func, 
#     layer_index = -2
# )

# Measuring the influence of data by removing records
influences1 = []
# influences2 = []
# influences3 = []

for index in range(0, X_train1.shape[0]):
    influences1.append(infl1.influence_remove_single(index))

# for index in range(0, X_train2.shape[0]):
#     influences2.append(infl2.influence_remove_single(index))

# for index in range(0, X_train3.shape[0]):
#     influences3.append(infl3.influence_remove_single(index))

# Note: The influence scores depend the current status of the model,
# even for the same model configuration with different random seed,
# the model may converge to different point in hyperspace, which will
# result in different score for the same records.
# Note2: The absolute value of the influece score is meaningless, but
# they are comparable with each other.

sum_influences1 = [sum_influences1[i] + influences1[i] for i in range(len(influences1))]
# sum_influences2 = [sum_influences2[i] + influences2[i] for i in range(len(influences2))]
# sum_influences3 = [sum_influences3[i] + influences3[i] for i in range(len(influences3))]

Epoch = 0, loss1 = 0.8360, accuracy=0.2731
Epoch = 10, loss1 = 6.2535, accuracy=0.7534
Epoch = 20, loss1 = 5.7801, accuracy=0.7522
Epoch = 30, loss1 = 5.7175, accuracy=0.7522
Epoch = 40, loss1 = 5.6669, accuracy=0.7522
Epoch = 50, loss1 = 5.5999, accuracy=0.7854
Epoch = 60, loss1 = 5.5881, accuracy=0.7898
Epoch = 70, loss1 = 5.5724, accuracy=0.7936
Epoch = 80, loss1 = 5.5662, accuracy=0.7939
Epoch = 90, loss1 = 5.5614, accuracy=0.7943
Epoch = 100, loss1 = 5.5583, accuracy=0.8075
Epoch = 110, loss1 = 5.5566, accuracy=0.7976
Epoch = 120, loss1 = 5.5550, accuracy=0.7989
Epoch = 130, loss1 = 5.5539, accuracy=0.8116
Epoch = 140, loss1 = 5.5548, accuracy=0.7966
Epoch = 150, loss1 = 5.5529, accuracy=0.7969
Epoch = 160, loss1 = 5.5524, accuracy=0.7969
Epoch = 170, loss1 = 5.5553, accuracy=0.7973
Epoch = 180, loss1 = 5.5498, accuracy=0.8009
Epoch = 190, loss1 = 5.5494, accuracy=0.8008
Epoch = 200, loss1 = 5.5561, accuracy=0.7972
Epoch = 210, loss1 = 5.5478, accuracy=0.8043
Epoch = 220, loss1 = 

In [ ]:
avg_influences1 = [val / 10 for val in sum_influences1]
avg_influences2 = [val / 10 for val in sum_influences2]
avg_influences3 = [val / 10 for val in sum_influences3]

In [ ]:
# Creating a dictionary of average influence scores with index
result1 = {idx: val for idx, val in enumerate(avg_influences1)}
print(list(result1.items())[:])
result2 = {idx: val for idx, val in enumerate(avg_influences2)}
print(list(result2.items())[:])
result3 = {idx: val for idx, val in enumerate(avg_influences3)}
print(list(result3.items())[:])

[(0, 0.00011568480958475768), (1, 0.037695386330699535), (2, -0.0019822391592659006), (3, 0.0), (4, 0.0009299880179344666), (5, 0.0), (6, -0.00018943950867105738), (7, -0.0014290242607274124), (8, 0.0), (9, 0.033375182589935076), (10, 0.0006367524452364979), (11, 0.0), (12, -0.003074676398830929), (13, -3.0439163354221813e-05), (14, -0.011405264456730917), (15, -0.0001719572258876187), (16, 0.017108532493863314), (17, 0.00039641028349625597), (18, 0.0004805679310263348), (19, -0.013875634280016818), (20, 0.0), (21, -1.6663003004343705e-05), (22, -0.00047533651988143096), (23, 0.030894858831936246), (24, 0.008870354903200647), (25, 0.0), (26, 0.0002767668682805311), (27, -0.004561978514621869), (28, -0.03886473207593755), (29, 0.0008757847539938862), (30, -0.0021456509130496613), (31, -0.001359550871923059), (32, -0.00021691549011861464), (33, 6.5270665561273005e-06), (34, 0.021798289166295497), (35, -0.07578868755827102), (36, 0.018246651749931474), (37, -0.0021106203876572064), (38, -

In [ ]:
print(avg_influences1)
print(avg_influences2)
print(avg_influences3)

[0.00011568480958475768, 0.037695386330699535, -0.0019822391592659006, 0.0, 0.0009299880179344666, 0.0, -0.00018943950867105738, -0.0014290242607274124, 0.0, 0.033375182589935076, 0.0006367524452364979, 0.0, -0.003074676398830929, -3.0439163354221813e-05, -0.011405264456730917, -0.0001719572258876187, 0.017108532493863314, 0.00039641028349625597, 0.0004805679310263348, -0.013875634280016818, 0.0, -1.6663003004343705e-05, -0.00047533651988143096, 0.030894858831936246, 0.008870354903200647, 0.0, 0.0002767668682805311, -0.004561978514621869, -0.03886473207593755, 0.0008757847539938862, -0.0021456509130496613, -0.001359550871923059, -0.00021691549011861464, 6.5270665561273005e-06, 0.021798289166295497, -0.07578868755827102, 0.018246651749931474, -0.0021106203876572064, -2.7073727561732005e-05, -0.034818802261599835, 0.00023332304363613931, 0.0, -0.00019254282908164543, 0.027643296391755872, -6.512818986660785e-05, 0.0001952210287195808, 0.03067337594108079, 0.0, -0.013047529523919858, -5.3

In [ ]:
print(influences1[0])

0.0011568480958475769


In [ ]:
print(influences1[1])

0.3769538633069953


In [ ]:
for i in influences1:
  print(i)

Streaming output truncated to the last 5000 lines.
-2.0856717230821373
0.0011131930723370968
0.24392824376139854
0.18795807915321047
-0.0005127868917885878
-0.0
-0.0
-0.48006257980034345
-0.0020041965129321674
-0.009552008584183152
0.6801120352870709
-0.2676916349131698
0.04927881150204432
-0.0009733892897386224
-0.04606094706387437
-0.0
0.03491887481869181
0.01077077067228801
-0.005614852337563106
-0.00014398745664258048
-0.039625247616883616
0.10526978291376171
-0.15748925799835936
0.0013473747401520132
0.012352241047649664
0.10645333776060231
-0.42805248906254445
0.055652245781736075
-0.03116657483152022
-0.0012754079893041168
-0.021734468993161466
-0.0010757829336129426
-0.0010554572507798128
-0.0
0.0035896336730086766
-0.0
-0.004085606301772568
-0.0
0.01685204835442819
0.016635106548683616
0.34847414767057444
-0.0
-0.0002833986089964571
0.004533214653381106
0.06865788245058325
0.22361034926878678
-0.08470184900182341
0.0006856385912345544
0.011130991668359345
0.0317501158653838
0.